In [1]:
import h2o
from h2o.automl import H2OAutoML
h2o.init(max_mem_size='200g')

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,2 hours 2 mins
H2O_cluster_timezone:,Europe/Berlin
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.1
H2O_cluster_version_age:,1 month and 4 days
H2O_cluster_name:,H2O_from_python_uwwfb_icmdd3
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,180.9 Gb
H2O_cluster_total_cores:,48
H2O_cluster_allowed_cores:,48
H2O_cluster_status:,"locked, healthy"


In [2]:
target_zone='zone_18'
runtime = 600

In [3]:
import pandas as pd

In [4]:
aml=h2o.upload_model(f'Frameworks/H2o/{target_zone}/{target_zone}_{runtime}')

In [5]:
test=h2o.import_file(f'Dataset/Zones/{target_zone}test.csv')

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [6]:
data_comlete = h2o.import_file(f'Dataset/{target_zone}.csv')
preds= data_comlete[target_zone]
#gap is the missing values
gap = data_comlete[range(39413,39432,1),:]

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [7]:
gap

datetime,year,month,day,hour,station_1,station_2,station_3,station_4,station_5,station_6,station_7,station_8,station_9,station_10,station_11,zone_9,target,workday
2008-06-30 05:00:00,2008,6,30,5,75,63,74,71,72,68,71,71,69,70,64,77196,nan,1
2008-06-30 06:00:00,2008,6,30,6,75,64,73,71,73,69,70,71,68,70,64,nan,nan,1
2008-06-30 07:00:00,2008,6,30,7,76,67,74,72,74,71,72,71,71,71,68,nan,nan,1
2008-06-30 08:00:00,2008,6,30,8,78,69,75,75,75,73,75,73,76,74,70,nan,nan,1
2008-06-30 09:00:00,2008,6,30,9,79,72,76,77,77,78,78,77,79,79,71,nan,nan,1
2008-06-30 10:00:00,2008,6,30,10,80,75,80,80,80,80,80,80,79,81,75,nan,nan,1
2008-06-30 11:00:00,2008,6,30,11,83,76,81,83,82,82,82,83,81,82,77,nan,nan,1
2008-06-30 12:00:00,2008,6,30,12,84,75,83,86,83,84,84,86,84,85,74,nan,nan,1
2008-06-30 13:00:00,2008,6,30,13,86,76,84,88,85,84,84,86,84,85,78,nan,nan,1
2008-06-30 14:00:00,2008,6,30,14,88,75,85,90,84,84,85,88,83,84,74,nan,nan,1


In [8]:
for x in range(0, 19):
    #row is the current hour in the loop starting with 2008-07-01 00:00:00
    row = gap[x,:]
    pred=aml.predict(row)
    gap[x+1,16]=pred[0,0]

last_value=pred[0,0]
print(last_value)
  

stackedensemble prediction progress: |████████████████████████████████████| 100%
stackedensemble prediction progress: |████████████████████████████████████| 100%
stackedensemble prediction progress: |████████████████████████████████████| 100%
stackedensemble prediction progress: |████████████████████████████████████| 100%
stackedensemble prediction progress: |████████████████████████████████████| 100%
stackedensemble prediction progress: |████████████████████████████████████| 100%
stackedensemble prediction progress: |████████████████████████████████████| 100%
stackedensemble prediction progress: |████████████████████████████████████| 100%
stackedensemble prediction progress: |████████████████████████████████████| 100%
stackedensemble prediction progress: |████████████████████████████████████| 100%
stackedensemble prediction progress: |████████████████████████████████████| 100%
stackedensemble prediction progress: |████████████████████████████████████| 100%
stackedensemble prediction p

In [9]:
test_data = pd.read_csv(f'Dataset/Zones/{target_zone}test.csv')
target = test_data['target']
preds = test_data[target_zone]
test = h2o.import_file(f'Dataset/Zones/{target_zone}test.csv')
#add the last prediction from the missing 19 value as the first target_column value
test[0,16]=last_value

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [10]:
target

0      76314.0
1      76377.0
2      79422.0
3      80346.0
4      92820.0
        ...   
162    75369.0
163    75600.0
164    76356.0
165    76839.0
166    76944.0
Name: target, Length: 167, dtype: float64

In [11]:
preds

0      75621.0
1      76314.0
2      76377.0
3      79422.0
4      80346.0
        ...   
162    63084.0
163    75369.0
164    75600.0
165    76356.0
166    76839.0
Name: zone_9, Length: 167, dtype: float64

In [12]:

for x in range(0, len(target)):
    row = test[x,:]
    print(x)
    temp=aml.predict(row)
    preds[x]=temp[0,0]
    test[x+1,16]=temp[0,0]


0
stackedensemble prediction progress: |████████████████████████████████████| 100%
1
stackedensemble prediction progress: |████████████████████████████████████| 100%
2
stackedensemble prediction progress: |████████████████████████████████████| 100%
3
stackedensemble prediction progress: |████████████████████████████████████| 100%
4
stackedensemble prediction progress: |████████████████████████████████████| 100%
5
stackedensemble prediction progress: |████████████████████████████████████| 100%
6
stackedensemble prediction progress: |████████████████████████████████████| 100%
7
stackedensemble prediction progress: |████████████████████████████████████| 100%
8
stackedensemble prediction progress: |████████████████████████████████████| 100%
9
stackedensemble prediction progress: |████████████████████████████████████| 100%
10
stackedensemble prediction progress: |████████████████████████████████████| 100%
11
stackedensemble prediction progress: |████████████████████████████████████| 100%
12

In [13]:
preds

0      70483.515659
1      70226.386351
2      69393.167953
3      69310.818176
4      67780.958517
           ...     
162    64888.663869
163    67020.669505
164    68701.020052
165    70238.705662
166    69966.543376
Name: zone_9, Length: 167, dtype: float64

In [14]:
target

0      76314.0
1      76377.0
2      79422.0
3      80346.0
4      92820.0
        ...   
162    75369.0
163    75600.0
164    76356.0
165    76839.0
166    76944.0
Name: target, Length: 167, dtype: float64

In [15]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import max_error

print(f'Metrics for H2o {target_zone}_{runtime}')
print('Min Load:', target.min())
print('Max Load:', target.max())
print('Average Load:', target.mean())
print('R2:', r2_score(target, preds))
print('MAE:', mean_absolute_error(target, preds))
print('MAPE:', mean_absolute_percentage_error(target, preds))
print('Max Error:', max_error(target, preds))

Metrics for H2o zone_9_14400
Min Load: 6174.0
Max Load: 93450.0
Average Load: 64101.17964071856
R2: 0.3916131310813379
MAE: 14209.641837386038
MAPE: 0.5511522264911954
Max Error: 54750.18695535511
